In [ ]:
# # !CUDA_VISIBLE_DEVICES=4,5,6,7
# !CUDA_VISIBLE_DEVICES=7,5,3

In [1]:
import os
import glob
import collections

import pickle
import sys
import glob
from pathlib import Path, PurePath
path = Path.cwd()
parent_path = path.parents[1]
sys.path.append('neuroformer')
sys.path.append('.')
sys.path.append('../')


import pandas as pd
import numpy as np
from einops import rearrange

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
from torch.utils.data.dataloader import DataLoader

import math
from torch.utils.data import Dataset

from attentionVis import AttentionVis
from trainer import Trainer, TrainerConfig
from utils import set_seed


from scipy import io as scipyio
from scipy.special import softmax
import skimage
import skvideo.io
from utils import print_full
from scipy.ndimage.filters import gaussian_filter, uniform_filter


import matplotlib.pyplot as plt
from utils import *
from visualize import *
set_plot_params()
%matplotlib inline
parent_path = os.path.dirname(os.path.dirname(os.getcwd())) + "/"


# for i in {1..10}; do python3 -m gather_atts.py; done

/tmp/ipykernel_1960918/1676056690.py:41: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter, uniform_filter
/tmp/ipykernel_1960918/1676056690.py:41: DeprecationWarning: Please use `uniform_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter, uniform_filter


In [2]:
""" 

-- DATA --
neuroformer/data/OneCombo3_V1AL/
df = response
video_stack = stimulus
DOWNLOAD DATA URL = https://drive.google.com/drive/folders/1jNvA4f-epdpRmeG9s2E-2Sfo-pwYbjeY?usp=sharing


"""

RESPONSE_PATH = "./data/NaturalMovie/response/NaturalStim_all.csv"
STIMULUS_PATH = "./data/NaturalMovie/stimulus/docuMovie"

# download data 

if not os.path.exists('./data/NaturalMovie'):
    import gdown
    id = 'https://drive.google.com/drive/folders/1_G7QHE_Pp5Tkp2qnaG8XZW0MlIxHCEQs?usp=sharing'
    gdown.download_folder(id=id, output='./data')

Retrieving folder list


Retrieving folder 19Rb_av93578cfFX8yuel-c1kN6RNrQcW response
Processing file 16W87GwZz1xd3cbMOA5-d46M3BWl7D_yr NaturalStim_all.csv
Processing file 1MIxZBSM_besxRkBnWtN0Tbim969-TEvj neuron_dict.pkl
Retrieving folder 1jAnbng1TvTiLo8nDERy6HOLwgqP-CF9Q stimulus
Retrieving folder 1Du8b4XFxcB9E3BzGh_M6G5Xx00meP8Bq docuMovie
Processing file 1Tu8vsUJIT5o5BvFmwSU-y-n_3Q7iKcoW A Perfect Blend of Five Flavors_clip1.tif
Processing file 1xIF-tbUSNMnaHXp-iDu2gVPdx_tqAm2E A Perfect Blend of Five Flavors_clip2.tif
Processing file 1Vk31IuQR1f2_P_PHI2ZiaNBLW3L-kIHr Beaver Documentary_clip1.tif
Processing file 1e8uuM1hwud5vA7viRMCXNAH-FokzhV-g Beaver Documentary_clip2.tif
Processing file 12kiFO4FJnLVh9OFT1UrYr46OquxF280m Beaver Documentary_clip3.tif
Processing file 1DdQbOrALyj9knMaonwSELOqjF_xUgf4L Lviv_clip1.tif
Processing file 1npinVkbe3eCBeRtMn8UJJgAvYICEhOca Plants Communicate_clip1.tif
Processing file 12-YghcSxkItt5HpStAaB9_BsWhU2DqqL videoplayback_clip1.tif
Processing file 1duMYlsEPTXO5Kll3u4EInv0k

Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=16W87GwZz1xd3cbMOA5-d46M3BWl7D_yr
To: /data5/antonis/neuroformer/data/response/NaturalStim_all.csv
100%|██████████| 1.14G/1.14G [00:10<00:00, 106MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1MIxZBSM_besxRkBnWtN0Tbim969-TEvj
To: /data5/antonis/neuroformer/data/response/neuron_dict.pkl
100%|██████████| 972M/972M [00:09<00:00, 107MB/s]  
Downloading...
From: https://drive.google.com/uc?id=1Tu8vsUJIT5o5BvFmwSU-y-n_3Q7iKcoW
To: /data5/antonis/neuroformer/data/stimulus/docuMovie/A Perfect Blend of Five Flavors_clip1.tif
100%|██████████| 382M/382M [00:03<00:00, 108MB/s]  
Downloading...
From: https://drive.google.com/uc?id=1xIF-tbUSNMnaHXp-iDu2gVPdx_tqAm2E
To: /data5/antonis/neuroformer/data/stimulus/docuMovie/A Perfect Blend of Five Flavors_clip2.tif
100%|██████████| 188M/188M [00:01<00:00, 110MB/s]  
Downloading...
From: https://drive.google.com/uc?id=1Vk31IuQR1f2_

In [3]:
# load config files
import yaml

base_path = f"./models/tensorboard/NaturalMovie"

with open(os.path.join(base_path, 'mconf.yaml'), 'r') as stream:
    mconf = yaml.full_load(stream)

with open(os.path.join(base_path, 'tconf.yaml'), 'r') as stream:
    tconf = yaml.full_load(stream)

# with open(os.path.join(base_path, 'dconf.yaml'), 'r') as stream:
#     dconf = yaml.full_load(stream)

import omegaconf
from omegaconf import OmegaConf

# open yaml as omegacong
mconf = OmegaConf.create(mconf)
tconf = OmegaConf.create(tconf)
# dconf = OmegaConf.create(dconf)

In [36]:
# R3D: (3 x T x H x W)

from SpikeVidUtils import image_dataset

# def nearest(n, x):
#   u = n % x > x // 2
#   return n + (-1)**(1 - u) * abs(x * u - n % x)

# vid_paths = sorted(glob.glob(STIMULUS_PATH + '/*.tif'))
# vid_list = [skimage.io.imread(vid)[::3] for vid in vid_paths]
# video_stack = [torch.nan_to_num(image_dataset(vid)).transpose(1, 0) for vid in vid_list]
# torch.save({k:v for k, v in enumerate(video_stack)}, '/data5/antonis/projects/neuroformer/data/SimNeu3D/NaturalMovie/stimulus/docuMovie.pt')

vs = torch.load("./data/NaturalMovie/stimulus/docuMovie.pt")
# video_stack = [vs[i] for i in range(len(vs))]
video_stack = dict()
for i in range(len(vs)):
    video_stack[i] = vs[i][0]

# plt.imshow(video_stack[0][0, 0])

In [5]:
frame_window = 1
window = 0.05
window_prev = 0.2
dt = 0.01
dt_frames = 0.05
visual_stim = True
DATASET = 'NaturalMovie'

In [6]:
max_window = max(window, window_prev)
dt_range = math.ceil(max_window / dt) + 1  # add first / last interval for SOS / EOS'
n_dt = [round(dt * n, 2) for n in range(dt_range)] + ['EOS'] + ['PAD']

In [7]:
from neuroformer.SpikeVidUtils import trial_df_real, make_intervals
from neuroformer.prepare_data import load_natmovie_real

# df['Interval'] = make_intervals(df, window)
# df['Interval_2'] = make_intervals(df, window_prev)
# df['Interval_dt'] = make_intervals(df, dt)
# df['Interval_dt'] = (df['Interval_dt'] - df['Interval'] + window).round(3)
# df = df.reset_index(drop=True)
# df.to_csv(f"data/SimNeu3D/NaturalMovie/response/NaturalStim_all_w:{window}_wp:{window_prev}.csv", index=False)
df = pd.read_csv(f"./data/NaturalMovie/response/NaturalStim_all_w:{window}_wp:{window_prev}.csv")

# data_dict = pickle.load(open("./data/NaturalMovie/response/neuron_dict.pkl", "rb"))

# df['Interval'] = make_intervals(df, window)
# df['real_interval'] = make_intervals(df, 0.05)
# df['Interval_2'] = make_intervals(df, window_prev)

max_window = max(window, window_prev)
dt_range = math.ceil(max_window / dt) + 1  # add first / last interval for SOS / EOS'
n_dt = [round(dt * n, 2) for n in range(dt_range)] + ['EOS'] + ['PAD']

In [69]:
save_path = "./data/NaturalMovie/response"
from neuroformer.SpikeVidUtils import neuron_dict
data_dict = neuron_dict(df)
dict_path = "./data/NaturalMovie/response/neuron_dict.pkl"
if not os.path.exists(dict_path):
    with open(os.path.join(save_path, dict_path), 'wb') as handle:
        pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

data_dict = pickle.load(open(dict_path, "rb"))

In [8]:
# int_trials = df.groupby(['Interval', 'Trial']).size()
# print(int_trials.mean())
# df.groupby(['Interval', 'Trial']).agg(['nunique'])
var_group = 'Interval'
# n_unique = len(df.groupby([var_group, 'Trial']).size())
# df.groupby([var_group, 'Trial']).size().nlargest(int(0.2 * n_unique))
groups = df.groupby([var_group, 'Trial']).size()
n_unique = len(groups)
groups.sort_values(ascending=False).nlargest(int(0.7 * n_unique))
# df.groupby([var_group, 'Trial']).size().sort_values(ascending=False).nsmallest(int(0.99 * n_unique))

Interval  Trial
80.70     721      49
          777      45
          772      44
          736      44
          665      43
                   ..
152.15    1322      2
31.40     776       2
152.15    1326      2
          1338      2
31.40     771       2
Length: 6744475, dtype: int64

In [9]:
var_group = 'Interval_2'
# n_unique = len(df.groupby([var_group, 'Trial']).size())
# df.groupby([var_group, 'Trial']).size().nlargest(int(0.2 * n_unique))
groups = df.groupby([var_group, 'Trial']).size()
n_unique = len(groups)
groups.sort_values(ascending=False).nlargest(int(0.7 * n_unique))

Interval_2  Trial
58.6        1115     124
            1117     123
            1009     122
            1101     121
            1052     121
                    ... 
392.2       481        8
81.4        1031       8
0.8         1419       8
185.8       407        8
198.4       1417       8
Length: 1897800, dtype: int64

In [70]:
## resnet3d feats
n_embd = 256
n_embd_frames = 64
conv_layer = True

n_frames = round(frame_window * 1/dt_frames)
kernel_size = [n_frames, 16, 16]
stride_size = [n_frames, 8, 8]
padding_size = [0, 0, 0]
frame_block_size = ((20 // kernel_size[0] * 64 * 112) // (n_embd_frames))
frame_feats = video_stack if visual_stim else None
frame_block_size = (20 * 64 * 112) // (n_embd_frames)
# frame_block_size = 560
id_block_size = 55
prev_id_block_size = 130
id_block_size = prev_id_block_size   # 95
block_size = frame_block_size + id_block_size + prev_id_block_size # frame_block_size * 2  # small window for faster training
frame_memory = 20   # how many frames back does model see
window = window

neurons = sorted(list(set(df['ID'])))
id_stoi = { ch:i for i,ch in enumerate(neurons) }
id_itos = { i:ch for i,ch in enumerate(neurons) }

neurons = [i for i in range(df['ID'].min(), df['ID'].max() + 1)]
feat_encodings = neurons + ['SOS'] + ['EOS'] + ['PAD']  # + pixels 
stoi = { ch:i for i,ch in enumerate(feat_encodings) }
itos = { i:ch for i,ch in enumerate(feat_encodings) }
stoi_dt = { ch:i for i,ch in enumerate(n_dt) }
itos_dt = { i:ch for i,ch in enumerate(n_dt) }




In [71]:
import random

r_split = 0.8
all_trials = sorted(df['Trial'].unique())
test_trials = [3, 5]
# train_trials = random.sample(all_trials, int(len(all_trials) * r_split))
train_trials = [i for i in all_trials if i not in test_trials]
test_trials = [3, 5]

train_data = df[df['Trial'].isin(train_trials)]
test_data = df[df['Trial'].isin(test_trials)]

In [72]:
from neuroformer.SpikeVidUtils import SpikeTimeVidData2


train_dataset = SpikeTimeVidData2(train_data, None, block_size, id_block_size, frame_block_size, prev_id_block_size, 
                                  window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats,
                                  pred=False, window_prev=window_prev, frame_window=frame_window,
                                  dt_frames=dt_frames, intervals=None, dataset=DATASET, data_dict=data_dict)
test_dataset = train_dataset.copy(test_data)

print(f'train: {len(train_dataset)}, test: {len(test_dataset)}')


Population Size:  363
ID Population Size:  363
DT Population Size:  23
train: 9619023, test: 15106


In [73]:
loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
iterable = iter(loader)

In [78]:
x, y = next(iterable)
# print(x['behavior'].shape, x['behavior_dt'].shape)
for k in x.keys():
    print(k, x[k].shape)

id_prev torch.Size([32, 130])
dt_prev torch.Size([32, 130])
pad_prev torch.Size([32])
id torch.Size([32, 130])
dt torch.Size([32, 130])
pad torch.Size([32])
interval torch.Size([32])
trial torch.Size([32])
frames torch.Size([32, 1, 20, 64, 112])
stimulus torch.Size([32])
cid torch.Size([32, 2])
pid torch.Size([32, 2])
f_idx torch.Size([32, 2])


In [84]:
from model_neuroformer import GPT, GPTConfig

layers = (mconf.n_state_layers, mconf.n_state_history_layers, mconf.n_stimulus_layers)   
max_epochs = 300
batch_size = round((32 * 3))
shuffle = True

title =  f'conv_{kernel_size}'

model_path = f"""./models/tensorboard/{DATASET}/{title}/sparse_f:{mconf.sparse_topk_frame}_id:{mconf.sparse_topk_id}/w:{window}_wp:{window_prev}/{6}_Cont:{mconf.contrastive}_window:{window}_f_window:{frame_window}_df:{dt}_blocksize:{id_block_size}_conv_{conv_layer}_shuffle:{shuffle}_batch:{batch_size}_sparse_({mconf.sparse_topk_frame}_{mconf.sparse_topk_id})_blocksz{block_size}_pos_emb:{mconf.pos_emb}_temp_emb:{mconf.temp_emb}_drop:{mconf.id_drop}_dt:{shuffle}_2.0_{max(stoi_dt.values())}_max{dt}_{layers}_{mconf.n_head}_{mconf.n_embd}.pt"""

model_conf = GPTConfig(train_dataset.population_size, block_size,    # frame_block_size
                        id_vocab_size=train_dataset.id_population_size,
                        frame_block_size=frame_block_size,
                        id_block_size=id_block_size,  # frame_block_size
                        prev_id_block_size=prev_id_block_size,
                        sparse_mask=False, p_sparse=None, 
                        sparse_topk_frame=None, sparse_topk_id=None, sparse_topk_prev_id=None,
                        n_dt=len(n_dt),
                        class_weights=None,
                        pretrain=False,
                        n_state_layers=mconf.n_state_layers, n_state_history_layers=mconf.n_state_history_layers,
                        n_stimulus_layers=mconf.n_stimulus_layers, self_att_layers=mconf.self_att_layers,
                        n_behavior_layers=0,
                        behavior_block_size=0,
                        predict_behavior=False,
                        n_head=mconf.n_head, n_embd=mconf.n_embd, 
                        contrastive=mconf.contrastive, clip_emb=1024, clip_temp=mconf.clip_temp,
                        conv_layer=conv_layer, 
                        kernel_size=kernel_size, stride_size=stride_size, padding_size=padding_size,
                        temp_emb=mconf.temp_emb, pos_emb=False,
                        wave_emb=True,
                        id_drop=0.35, im_drop=0.35, b_drop=0.45,
                        window=window, window_prev=window_prev, frame_window=frame_window, dt=dt,
                        neurons=neurons, stoi_dt=stoi_dt, itos_dt=itos_dt, n_embd_frames=n_embd_frames,
                        ignore_index_id=stoi['PAD'], ignore_index_dt=stoi_dt['PAD'])  # 0.35

model = GPT(model_conf)
# model.load_state_dict(torch.load(model_path))


573440
256 23


In [80]:
preds, features, loss = model(x, y)

In [85]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

tconf = TrainerConfig(max_epochs=max_epochs, batch_size=batch_size, learning_rate=2e-4, 
                    num_workers=4, lr_decay=True, patience=3, warmup_tokens=8e7, 
                    decay_weights=True, weight_decay=0.2, shuffle=shuffle,
                    final_tokens=len(train_dataset)*(id_block_size) * (max_epochs),
                    clip_norm=1.0, grad_norm_clip=1.0,
                    dataset='higher_order', mode='predict',
                    block_size=train_dataset.block_size,
                    id_block_size=train_dataset.id_block_size,
                    show_grads=False, plot_raster=False,
                    ckpt_path=model_path, no_pbar=False, 
                    dist=False, save_every=1000)

trainer = Trainer(model, train_dataset, test_dataset, tconf, model_conf)
trainer.train()


not decaying: temp_emb_behavior.temp_emb.0.weight
not decaying: temp_emb_behavior.temp_emb.0.bias
not decaying: temp_emb_behavior.temp_emb.2.weight
not decaying: temp_emb_behavior.temp_emb.2.bias
weight_decay: 0.2


  0%|          | 0/100199 [03:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from neuroformer.utils import predict_raster_recursive_time_auto, process_predictions

model.load_state_dict(torch.load(model_path))

results_dict = dict()
df_pred = None
df_true = None

top_p = 0.75
top_p_t = 0.75
temp = 1.25
temp_t = 1.25
max_interval = 32

trials = test_data['Trial'].unique()
for trial in trials:   
        print(f"Trial: {trial}")
        df_trial = df[(df['Trial'] == trial) & (df['Interval'] <= max_interval)]
        trial_dataset = SpikeTimeVidData2(df_trial,  None, block_size, id_block_size, frame_block_size, prev_id_block_size, 
                                  window, dt, frame_memory, stoi, itos, neurons, stoi_dt, itos_dt, frame_feats,
                                  pred=False, window_prev=window_prev, frame_window=frame_window,
                                  dt_frames=dt_frames, intervals=None, dataset=DATASET,
                                  window_behavior=window_behavior)
        results_trial = predict_raster_recursive_time_auto(model, trial_dataset, window, window_prev, stoi, itos_dt, itos=itos, 
                                                           sample=True, top_p=top_p, top_p_t=top_p_t, temp=temp, temp_t=temp_t, 
                                                           frame_end=0, get_dt=True, gpu=False, pred_dt=True)
        # results_trial = predict_raster_hungarian(model, loader, itos_dt, top_p=0.75, temp=1)
        # print(f"MAX ID ---- {sorted(results_trial['ID'].unique()[-10])}")
        df_trial_pred, df_trial_true = process_predictions(results_trial, stoi, itos, window)
        print(f"pred: {df_trial_pred.shape}, true: {df_trial_true.shape}" )
        if df_pred is None:
            df_pred = df_trial_pred
            df_true = df_trial_true
        else:
            df_pred = pd.concat([df_pred, df_trial_pred])
            df_true = pd.concat([df_true, df_trial_true])


from neuroformer.analysis import compute_scores
scores = compute_scores(df[df['Trial'].isin(trials)], df_pred)
print(scores)
print(f"pred: {len(df_pred)}, true: {len(df_true)}" )


dir_name = os.path.dirname(model_path)
model_name = os.path.basename(model_path)
df_pred.to_csv(os.path.join(dir_name, F'df_pred_.csv'))


In [ ]:
from analysis import get_rates_trial, calc_corr_psth

df_1 = df[df['Trial'].isin(trials) & (df['Interval'] <= max_interval)]
df_pred_full = df_pred

window_pred = 0.5
window_pred = window if window_pred is None else window_pred
intervals = np.array(sorted(set(df['Interval'].unique()) & set(df['Interval'].unique())))
labels = np.array([round(window_pred + window_pred*n, 2) for n in range(0, int(max(df_pred_full['Interval']) / window_pred))])
ids = sorted(set(df['ID'].unique()) & set(df['ID'].unique()))

rates_pred = get_rates_trial(df_pred_full, labels)
rates_1 = get_rates_trial(df_1, labels)

top_corr_pred = calc_corr_psth(rates_pred, rates_1)

In [ ]:
"""

Evaluate results

"""

from neuroformer.visualize import *


len_pred, len_true = len(df_pred_full), len(df_1)
print(f"len_pred: {len_pred}, len_true: {len_true}")

accuracy = get_accuracy(df_pred, df_1)
pred_scores = compute_scores(df_1, df_pred_full)

print(f"pred: {pred_scores}")

n_bins = 30
set_plot_white()
plt.figure(figsize=(10, 10), facecolor='white')
plt.title(f'PSTH Correlations (V1 + AL) {title}', fontsize=25)
plt.ylabel('Count (n)', fontsize=25)
plt.xlabel('Pearson r', fontsize=25)
# plt.hist(top_corr_real_2, label='real - real3', alpha=0.6)
plt.hist(top_corr_pred, label='real - simulated', alpha=0.6, bins=30)
plt.legend(fontsize=20)

dir_name = os.path.dirname(model_path)

top_p = 0
save_title = f'_top_p{top_p}'
plt.savefig(os.path.join(dir_name, F'psth_corr_{save_title}_.svg'))
df_pred.to_csv(os.path.join(dir_name, F'df_pred_{save_title}_.csv'))

plot_distribution(df_1, df_pred, save_path=os.path.join(dir_name, F'psth_dist_.svg'))

total_scores = dict()
total_scores['pred'] = pred_scores

print(f"model: {title}")


In [ ]:
x, y = next(iterable)

T = len(x['id'])
P = x['pad'] - 1
T_prev = len(x['id_prev'])
P_prev = x['pad_prev'] - 4

iv = float(x['interval'])

xid = x['id'][: T - P]
xid = [itos[int(i)] for i in xid]

xid_prev = x['id_prev'][: T_prev - P_prev]
xid_prev = [itos[int(i)] for i in xid_prev]

print(f"iv: {iv}, ix+window: {iv + window} pid: {x['pid']} cid: {x['cid']}")
print(f"x: {xid}")

print(f"xid_prev: {xid_prev}")

tdiff = 0
t_var = 'Time' # 'Interval'
int_var = 'cid'
# df[(df[t_var] >= iv - tdiff) & (df[t_var] <= iv + (window + tdiff)) & (df['Trial'] == int(x['trial']))]
# df[(df[t_var] >= float(x[int_var][0]) - tdiff) & (df[t_var] <= float(x[int_var][1] + tdiff)) & (df['Trial'] == int(x['trial']))]
df[(df[t_var] > float(x[int_var][0]) - tdiff) & (df[t_var] <= float(x['cid'][1] + tdiff)) & (df['Trial'] == int(x['trial']))]

# t_var = 'Time' # 'Interval'
# int_var = 'pid'
# df[(df[t_var] > round(float(x[int_var][0]), 2) - tdiff) & (df[t_var] <= round(float(x[int_var][1]), 2)) & (df['Trial'] == int(x['trial']))]


